In [2]:
import sys

print(sys.path)
sys.path.append("../src")

import RSC as RSC
import util as util
import NUDGE as nudge

['/data2/wjlee/sw/conda/3/envs/RCT/lib/python39.zip', '/data2/wjlee/sw/conda/3/envs/RCT/lib/python3.9', '/data2/wjlee/sw/conda/3/envs/RCT/lib/python3.9/lib-dynload', '', '/home/wjlee/.local/lib/python3.9/site-packages', '/data2/wjlee/sw/conda/3/envs/RCT/lib/python3.9/site-packages', '../src']


In [3]:
PARAMS = util.load_yaml('../input/params.yaml')


In [10]:
PARAMS

{'network_file': './input/networks/toy.bnet',
 'target': 'Y',
 'use_mutations': False,
 'mutations': {},
 'verbose_rsc': False,
 'print_freq': 10,
 'max_recursion': 100,
 'approx': True,
 'max_variables': 12,
 'approx_init_fns': True,
 'max_init_variables': 8,
 'activity_samples': 100,
 'verbose_nudge': True,
 'ensemble_repeats': 8,
 'analysis_order': 3,
 'measure_robustness': True,
 'find_mechanism': True,
 'plot_mechanism': True,
 'controller': 'A',
 'network_dir': './input/networks/large/',
 'targets_csv': './input/targets_large_4targets.csv',
 'output_folder': './output/',
 'skip_reduced_networks': False,
 'cutoff': 0,
 'include_NUDGE': 1,
 'include_MONTE_CARLO': 1,
 'max_monte_carlo_controllers': 1,
 'monte_carlo_samples': 1000,
 'monte_carlo_time_steps': 1000,
 'include_LDOI': 1,
 'trim_ldoi': 1,
 'ldoi_pair_cap': 64,
 'include_MEAN-FIELD': 1,
 'trim_mf': 1,
 'mf_pair_cap': 100,
 'meanfield_epsilon': 0.001,
 'meanfield_time_steps': 100}

In [4]:
import numpy as np
import pandas as pd

def get_heterogeneous_path_list_and_highest_order(mechanism_data):
    heterogeneous_path_list = []
    highest_order = 1
    if mechanism_data != None:
        for target in mechanism_data:
            for k in mechanism_data[target]:
                if k == 'graph': continue
                for clause, order in mechanism_data[target][k]:
                    if order > 1:
                        heterogeneous_path_list.append(clause)
                        highest_order = max(highest_order, order)
    return heterogeneous_path_list, highest_order


In [11]:
import multiprocessing as mp
from typing import Any, Dict, Tuple

def _child_target(params: Dict[str, Any], out_q: mp.Queue) -> None:
    try:
        results = nudge.control(params)
        #print(1)
        out_q.put({"success": 1, "results": results})
        #print(2)
    except Exception:
        out_q.put({"success": 0, "results": "error"})
        #print(3)

def run_single_trial(params: Dict[str, Any], timeout_sec: int = 300) -> Tuple[int, Any]:
    ctx = mp.get_context()
    out_q: mp.Queue = ctx.Queue()
    p = ctx.Process(target=_child_target, args=(params, out_q))
    p.start()
    p.join(timeout=timeout_sec)

    # 1) still alive after timeout_sec
    if p.is_alive():
        p.terminate()
        p.join()
        return 0, 'timeout'

    # 2) process done, but no result → error
    try:
        msg = out_q.get_nowait()
    except Exception:
        return 0, 'error'

    success = int(msg.get("success", 0))
    fin = msg.get("results", None)

    # 3) success == 1
    if success == 1:
        return 1, fin

    # 4) success != 1 → error
    return 0, 'error'


### Biological networks

In [6]:
# read network names and target names
import pandas as pd

df = pd.read_csv('../input/targets_all_8targets.csv')
network_names = []
for line in df.to_numpy():
    network_names.append(line[0])

df['Network'] = network_names

In [7]:
network_target = []
for line in df.to_numpy():
    network_target.append((line[0], line[1]))


In [8]:
save_file = '../output/heterogeneous_paths/biological_network_heterogeneous_paths_and_highest_order.tsv'

f = open(save_file, 'w')
line = 'Network\tTarget\tExistence\tHeterogeneous_paths\tHighest_order\n'
f.write(line)
f.close()

In [12]:
trial = 1

for i in range(len(network_target)):
    network_file_name = '../input/networks/all/' + network_target[i][0]
    params = PARAMS.copy()
    params['target'] = network_target[i][1]
    params['network_file'] = network_file_name
    params['analysis_order'] = 4
    params['complement_constrain'] = True
    params['ensemble_repeats'] = 4

    heterogeneous_path_list = []

    highest_highest_order = 1
    for j in range(trial):
        success, results = run_single_trial(params)
        if type(results) == str:
            continue
        heterogeneous_path_list, highest_order = get_heterogeneous_path_list_and_highest_order(results['mechanism data'])
        highest_highest_order = max(highest_highest_order, highest_order)

    heterogeneous_path_list = list(np.unique(heterogeneous_path_list))

    save_line = '{}\t{}\t{}\t'.format(network_target[i][0], network_target[i][1], int(len(heterogeneous_path_list) > 0))
    for heterogeneous_path in heterogeneous_path_list:
        save_line += heterogeneous_path
        save_line += ','
    save_line += '\t{}'.format(highest_highest_order)

    save_line += '\n'

    f = open(save_file, 'a')
    f.write(save_line)
    f.close()


In [13]:
save_file = '../output/heterogeneous_paths/biological_network_heterogeneous_paths_and_highest_order.tsv'

neutral_driver_df = pd.read_csv(save_file, sep='\t')

In [14]:
highest_order_df = []
for network_name in np.unique(neutral_driver_df['Network']):
    highest_order = neutral_driver_df.loc[neutral_driver_df['Network']==network_name]['Highest_order'].max()
    highest_order_df.append([network_name, highest_order])

highest_order_df = pd.DataFrame(highest_order_df, columns=['Network', 'Highest_order'])

print(sum(highest_order_df['Highest_order']==1))
print(sum(highest_order_df['Highest_order']==2))
print(sum(highest_order_df['Highest_order']==3))
print(sum(highest_order_df['Highest_order']==4))


68
19
13
20


### Rewired biological networks

In [16]:
# rewired biological networks
# 1) nodes` order is rewired
# 2) logics are rewired
# 3) nodes and logics are all rewired
# while maintaining degree distribution

In [18]:
def read_nodes_logics_primes(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()

    bnet = '\n'
    nodes = []
    logics = {}
    for line in lines:
        line = line.replace('\n', '').replace('\t', '')
        nodes.append(line.split(',')[0])
        logics[line.split(',')[0]] = line.split(',')[1].replace('!', '! ')
        bnet = bnet + line.replace('!', '! ') + '\n'

    #print(bnet)
    #primes = bnet2primes(bnet)
    #primes = dict([(n, [np.array(primes[n][0]), np.array(primes[n][1])]) for n in primes])

    return nodes, logics


import networkx as nx


def get_BN_structure(logic_dict):
    GRN_structure = nx.DiGraph()
    for node in logic_dict:
        splited = logic_dict[node].replace('  ', ' ').replace('(', '( ').replace(')', ' )').split(' ')
        for i in range(len(splited)):
            if splited[i] not in ['', '(', ')', '&', '|', '~', '!']:
                if i == 0 or '!' == splited[i - 1]:
                    GRN_structure.add_edge(splited[i], node, weight=-1)
                else:
                    GRN_structure.add_edge(splited[i], node, weight=1)
    return GRN_structure


In [ ]:
# 1) node rewired

random_seed = 777
np.random.seed(random_seed)

for network_name in np.unique(network_names):
    network_file_name = '../input/networks/all/' + network_name

    nodes, logics = read_nodes_logics_primes(network_file_name)

    rewired_nodes_order = nodes[:]
    np.random.shuffle(rewired_nodes_order)

    node_rewired_network = {}
    for i in range(len(nodes)):
        node_rewired_network[rewired_nodes_order[i]] = logics[nodes[i]]

    f = open('../input/networks/randomized/node_rewired/' + network_name.replace('.bnet', '_node_rewired_{}.bnet'.format(random_seed)), 'w')
    for node in node_rewired_network:
        f.write('{}, {}\n'.format(node, node_rewired_network[node]))
    f.close()

In [ ]:
# 2) logic rewired

from random_networks import random_logic_node

random_logic_params = {
		'approx': False,
		'monotonic': True, # must be False for 'TT' method
		'method': 'kTTpartial',
		'max_clauses': None, #only used for 'kmaxTT', 'kTT', and 'kTTpartial' methods
		'enforce_all_edges': True
	}

random_seed = 777
np.random.seed(random_seed)

for network_name in np.unique(network_names):
    network_file_name = '../input/networks/all/' + network_name

    nodes, logics = read_nodes_logics_primes(network_file_name)

    GRN_structure = get_BN_structure(logics)

    logic_rewired_network = {}
    for node in nodes:
        input_nodes = [e[0] for e in GRN_structure.in_edges(node)]
        random_logic_params['max_clauses'] = len(input_nodes)
        fn = random_logic_node(random_logic_params, node, input_nodes)
        fn_str = ''
        for clause in fn:
            clause_str = ''
            for var in clause:
                clause_str += '{} & '.format(var)
            clause_str = clause_str[:-3]
            if len(clause) > 1:
                clause_str = '(' + clause_str + ')'
            fn_str += '{} | '.format(clause_str)
        fn_str = fn_str[:-3]

        logic_rewired_network[node] = fn_str

    f = open(
        '../input/network/logic_rewired/' + network_name.replace('.bnet', '_logic_rewired_{}.bnet'.format(random_seed)),
        'w')
    for node in logic_rewired_network:
        f.write('{}, {}\n'.format(node, logic_rewired_network[node]))
    f.close()

In [ ]:
# 3) all rewired

from random_networks import random_logic_node

random_logic_params = {
		'approx': False,
		'monotonic': True, # must be False for 'TT' method
		'method': 'kTTpartial',
		'max_clauses': None, #only used for 'kmaxTT', 'kTT', and 'kTTpartial' methods
		'enforce_all_edges': True
	}

random_seed = 777
np.random.seed(random_seed)

for network_name in np.unique(network_names):
    network_file_name = '../input/networks/all/' + network_name

    nodes, logics = read_nodes_logics_primes(network_file_name)

    GRN_structure = get_BN_structure(logics)

    all_rewired_network = {}
    for node in nodes:
        input_nodes = [e[0] for e in GRN_structure.in_edges(node)]
        random_logic_params['max_clauses'] = len(input_nodes)
        rewired_input_nodes = np.random.choice(GRN_structure.nodes(), len(input_nodes), replace=False)
        fn = random_logic_node(random_logic_params, node, rewired_input_nodes)
        fn_str = ''
        for clause in fn:
            clause_str = ''
            for var in clause:
                clause_str += '{} & '.format(var)
            clause_str = clause_str[:-3]
            if len(clause) > 1:
                clause_str = '(' + clause_str + ')'
            fn_str += '{} | '.format(clause_str)
        fn_str = fn_str[:-3]

        all_rewired_network[node] = fn_str

    f = open(
        '../input/networks/randomized/both_rewired/' + network_name.replace('.bnet', '_all_rewired_{}.bnet'.format(random_seed)),
        'w')
    for node in all_rewired_network:
        f.write('{}, {}\n'.format(node, all_rewired_network[node]))
    f.close()

In [19]:
import pandas as pd

random_seed = 777

df = pd.read_csv('../input/targets_all_8targets.csv')

node_rewired_network_names = []
logic_rewired_network_names = []
all_rewired_network_names = []
for line in df.to_numpy():
    node_rewired_network_names.append(line[0].replace('.bnet', '_node_rewired_{}.bnet'.format(random_seed)))
    logic_rewired_network_names.append(line[0].replace('.bnet', '_logic_rewired_{}.bnet'.format(random_seed)))
    all_rewired_network_names.append(line[0].replace('.bnet', '_all_rewired_{}.bnet'.format(random_seed)))

df['Network'] = network_names


In [20]:
node_rewired_network_target = []
logic_rewired_network_target = []
all_rewired_network_target = []
for i, line in enumerate(df.to_numpy()):
    node_rewired_network_target.append((node_rewired_network_names[i], line[1]))
    logic_rewired_network_target.append((logic_rewired_network_names[i], line[1]))
    all_rewired_network_target.append((all_rewired_network_names[i], line[1]))


In [ ]:
# 1) node rewired

In [ ]:
# refresh output file
save_file = '../output/RSC_neutral_motif/node_rewired_biological_network_heterogeneous_paths_and_highest_order.tsv'

f = open(save_file, 'w')
line = 'Network\tTarget\tExistence\tHeterogeneous_paths\tHighest_order\n'
f.write(line)
f.close()

In [ ]:
trial = 1

for i in range(len(node_rewired_network_target)):
    print(i, node_rewired_network_target[i])

    network_file_name = '../input/networks/randomized/node_rewired/' + node_rewired_network_target[i][0]
    params = PARAMS.copy()
    params['target'] = node_rewired_network_target[i][1]
    params['network_file'] = network_file_name
    params['analysis_order'] = 4
    params['complement_constrain'] = True
    params['ensemble_repeats'] = 4

    heterogeneous_path_list = []

    highest_highest_order = 1
    for j in range(trial):
        success, results = run_single_trial(params)
        if type(results) == str:
            continue
        heterogeneous_path_list, highest_order = get_heterogeneous_path_list_and_highest_order(results['mechanism data'])
        highest_highest_order = max(highest_highest_order, highest_order)

    heterogeneous_path_list = list(np.unique(heterogeneous_path_list))

    save_line = '{}\t{}\t{}\t'.format(node_rewired_network_target[i][0], node_rewired_network_target[i][1], int(len(heterogeneous_path_list) > 0))
    for heterogeneous_path in heterogeneous_path_list:
        save_line += heterogeneous_path
        save_line += ','
    save_line += '\t{}'.format(highest_highest_order)

    save_line += '\n'

    f = open(save_file, 'a')
    f.write(save_line)
    f.close()


In [21]:
save_file = '../output/heterogeneous_paths/node_rewired_biological_network_heterogeneous_paths_and_highest_order.tsv'

node_rewired_neutral_driver_df = pd.read_csv(save_file, sep='\t')

In [22]:
node_rewired_exist_networks = np.unique(node_rewired_neutral_driver_df.loc[node_rewired_neutral_driver_df['Existence']==1]['Network'])


In [23]:
print(len(node_rewired_exist_networks), len(np.unique(node_rewired_neutral_driver_df['Network'])))


26 120


In [24]:
highest_order_df = []
for network_name in np.unique(node_rewired_neutral_driver_df['Network']):
    highest_order = node_rewired_neutral_driver_df.loc[node_rewired_neutral_driver_df['Network']==network_name]['Highest_order'].max()
    highest_order_df.append([network_name, highest_order])

highest_order_df = pd.DataFrame(highest_order_df, columns=['Network', 'Highest_order'])

print(sum(highest_order_df['Highest_order']==1))
print(sum(highest_order_df['Highest_order']==2))
print(sum(highest_order_df['Highest_order']==3))
print(sum(highest_order_df['Highest_order']==4))


94
17
6
3


In [ ]:
# 2) logic rewired

In [ ]:
# refresh output file
save_file = '../output/heterogeneous_pahts/logic_rewired_biological_network_heterogeneous_paths_and_highest_order.tsv'

f = open(save_file, 'w')
line = 'Network\tTarget\tExistence\tHeterogeneous_paths\tHighest_order\n'
f.write(line)
f.close()

In [ ]:
trial = 1

for i in range(len(logic_rewired_network_target)):
    print(i, logic_rewired_network_target[i])

    network_file_name = '../input/networks/randomized/logic_rewired/' + logic_rewired_network_target[i][0]
    params = PARAMS.copy()
    params['target'] = logic_rewired_network_target[i][1]
    params['network_file'] = network_file_name
    params['analysis_order'] = 4
    params['complement_constrain'] = True
    params['ensemble_repeats'] = 4

    heterogeneous_path_list = []

    highest_highest_order = 1
    for j in range(trial):
        success, results = run_single_trial(params)
        if type(results) == str:
            continue
        heterogeneous_path_list, highest_order = get_heterogeneous_path_list_and_highest_order(results['mechanism data'])
        highest_highest_order = max(highest_highest_order, highest_order)

    heterogeneous_path_list = list(np.unique(heterogeneous_path_list))

    save_line = '{}\t{}\t{}\t'.format(logic_rewired_network_target[i][0], logic_rewired_network_target[i][1], int(len(heterogeneous_path_list) > 0))
    for heterogeneous_path in heterogeneous_path_list:
        save_line += heterogeneous_path
        save_line += ','
    save_line += '\t{}'.format(highest_highest_order)

    save_line += '\n'

    f = open(save_file, 'a')
    f.write(save_line)
    f.close()

In [25]:
save_file = '../output/heterogeneous_paths/logic_rewired_biological_network_heterogeneous_paths_and_highest_order.tsv'

logic_rewired_neutral_driver_df = pd.read_csv(save_file, sep='\t')


In [26]:
logic_rewired_exist_networks = np.unique(logic_rewired_neutral_driver_df.loc[logic_rewired_neutral_driver_df['Existence']==1]['Network'])


In [27]:
print(len(logic_rewired_exist_networks), len(np.unique(logic_rewired_neutral_driver_df['Network'])))


40 120


In [28]:
highest_order_df = []
for network_name in np.unique(logic_rewired_neutral_driver_df['Network']):
    highest_order = logic_rewired_neutral_driver_df.loc[logic_rewired_neutral_driver_df['Network']==network_name]['Highest_order'].max()
    highest_order_df.append([network_name, highest_order])

highest_order_df = pd.DataFrame(highest_order_df, columns=['Network', 'Highest_order'])

print(sum(highest_order_df['Highest_order']==1))
print(sum(highest_order_df['Highest_order']==2))
print(sum(highest_order_df['Highest_order']==3))
print(sum(highest_order_df['Highest_order']==4))


80
19
13
8


In [ ]:
# 3) all rewired

In [ ]:
# refresh output file
save_file = '../output/heterogeneous_paths/all_rewired_biological_network_heterogeneous_paths_and_highest_order_260116.tsv'

f = open(save_file, 'w')
line = 'Network\tTarget\tExistence\tHeterogeneous_paths\tHighest_order\n'
f.write(line)
f.close()

In [ ]:
trial = 1

for i in range(len(all_rewired_network_target)):

    print(i, all_rewired_network_target[i])

    network_file_name = '../input/networks/randomized/both_rewired/' + all_rewired_network_target[i][0]
    params = PARAMS.copy()
    params['target'] = all_rewired_network_target[i][1]
    params['network_file'] = network_file_name
    params['analysis_order'] = 4
    params['complement_constrain'] = True
    params['ensemble_repeats'] = 4

    heterogeneous_path_list = []

    highest_highest_order = 1
    for j in range(trial):
        success, results = run_single_trial(params)
        if type(results) == str:
            continue
        heterogeneous_path_list, highest_order = get_heterogeneous_path_list_and_highest_order(results['mechanism data'])
        highest_highest_order = max(highest_highest_order, highest_order)

    heterogeneous_path_list = list(np.unique(heterogeneous_path_list))

    save_line = '{}\t{}\t{}\t'.format(all_rewired_network_target[i][0], all_rewired_network_target[i][1], int(len(heterogeneous_path_list) > 0))
    for heterogeneous_path in heterogeneous_path_list:
        save_line += heterogeneous_path
        save_line += ','
    save_line += '\t{}'.format(highest_highest_order)

    save_line += '\n'

    f = open(save_file, 'a')
    f.write(save_line)
    f.close()

In [30]:
save_file = '../output/heterogeneous_paths/all_rewired_biological_network_heterogeneous_paths_and_highest_order.tsv'

all_rewired_neutral_driver_df = pd.read_csv(save_file, sep='\t')


In [31]:
all_rewired_exist_networks = np.unique(all_rewired_neutral_driver_df.loc[all_rewired_neutral_driver_df['Existence']==1]['Network'])


In [32]:
print(len(all_rewired_exist_networks), len(np.unique(all_rewired_neutral_driver_df['Network'])))


16 120


In [33]:
highest_order_df = []
for network_name in np.unique(all_rewired_neutral_driver_df['Network']):
    highest_order = all_rewired_neutral_driver_df.loc[all_rewired_neutral_driver_df['Network']==network_name]['Highest_order'].max()
    highest_order_df.append([network_name, highest_order])

highest_order_df = pd.DataFrame(highest_order_df, columns=['Network', 'Highest_order'])

print(sum(highest_order_df['Highest_order']==1))
print(sum(highest_order_df['Highest_order']==2))
print(sum(highest_order_df['Highest_order']==3))
print(sum(highest_order_df['Highest_order']==4))


104
8
5
3


### Random Boolean networks

In [34]:
import sys
print(sys.path)
sys.path.append('../src/random_networks_SF')
from Boolnet_form_scalefreeBNgenreator import main

['/data2/wjlee/sw/conda/3/envs/RCT/lib/python39.zip', '/data2/wjlee/sw/conda/3/envs/RCT/lib/python3.9', '/data2/wjlee/sw/conda/3/envs/RCT/lib/python3.9/lib-dynload', '', '/home/wjlee/.local/lib/python3.9/site-packages', '/data2/wjlee/sw/conda/3/envs/RCT/lib/python3.9/site-packages', '../src', '../src', '../input/src/random_networks_SF']


In [35]:
# totally random 120 networks
min_lscc_size = 11
max_lscc_size = 20
num_networks = 12


random_seed = 777
np.random.seed(random_seed)

trial = 0
network_name_list = []
for lscc_size in range(min_lscc_size, max_lscc_size+1):
    num = 0
    while 1:
        trial += 1

        max_degree = np.random.choice(list(range(5, min(11, lscc_size+1))))
        formatAttr, formatNormal, indegreeTotalLength = main(lscc_size, 1, max_degree, random_seed = int('{}{}'.format(trial, random_seed)))  # (node개수, 최소 degree 개수, 최대 degree 개수)

        bnet = formatNormal
        logic_dict = {}
        for line in bnet.split('\n')[:-1]:
            logic_dict[line.split(',')[0]] = line.split(', ')[1]
        GRN_structure = get_BN_structure(logic_dict)
        sccs = nx.strongly_connected_components(GRN_structure)
        if len(list(sccs)) != 1:
            continue

        num += 1


        f = open('../input/networks/randomized/random_networks/RBN_{}_{}_{}_{}.bn'.format(lscc_size, trial, random_seed, num), 'w')
        for line in bnet:
            f.write(line)
        f.close()

        network_name_list.append('RBN_{}_{}_{}_{}.bn'.format(lscc_size, trial, random_seed, num))

        if num == num_networks:
            break


In [36]:
pd.DataFrame(network_name_list).to_csv('../input/networks/randomized/random_networks/network_name_list.csv')


In [37]:
network_name_list = pd.read_csv('../input/networks/randomized/random_networks/network_name_list.csv')['0'].values


In [38]:
# choose 8 random output nodes
random_seed = 777
np.random.seed(random_seed)

RBN_network_target = []

for network_name in network_name_list:
    f = open('../input/networks/randomized/random_networks/'+network_name, 'r')
    lines = f.readlines()
    f.close()
    nodes = [line.split(',')[0] for line in lines]
    random_output_nodes = np.random.choice(nodes, 8, replace=False)
    for output_node in random_output_nodes:
        RBN_network_target.append([network_name, output_node])
        RBN_network_target.append([network_name, '!'+output_node])


In [ ]:
# refresh output file
RBN_save_file = '../output/heterogeneous_paths/RBN_heterogeneous_paths_and_highest_order.tsv'

f = open(RBN_save_file, 'w')
line = 'Network\tTarget\tExistence\tHeterogeneous_paths\tHighest_order\n'
f.write(line)
f.close()

In [ ]:
trial = 1

for i in range(len(RBN_network_target)):
    print(i, RBN_network_target[i])

    network_file_name = '../input/networks/randomized/random_networks/' + RBN_network_target[i][0]
    params = PARAMS.copy()
    params['target'] = RBN_network_target[i][1]
    params['network_file'] = network_file_name
    params['analysis_order'] = 4
    params['complement_constrain'] = True
    params['ensemble_repeats'] = 4

    heterogeneous_path_list = []

    highest_highest_order = 1
    for j in range(trial):
        success, results = run_single_trial(params)
        if type(results) == str:
            continue
        heterogeneous_path_list, highest_order = get_heterogeneous_path_list_and_highest_order(results['mechanism data'])
        highest_highest_order = max(highest_highest_order, highest_order)

    heterogeneous_path_list = list(np.unique(heterogeneous_path_list))

    save_line = '{}\t{}\t{}\t'.format(RBN_network_target[i][0], RBN_network_target[i][1], int(len(heterogeneous_path_list) > 0))
    for heterogeneous_path in heterogeneous_path_list:
        save_line += heterogeneous_path
        save_line += ','
    save_line += '\t{}'.format(highest_highest_order)

    save_line += '\n'

    f = open(RBN_save_file, 'a')
    f.write(save_line)
    f.close()


In [39]:
RBN_save_file = '../output/heterogeneous_paths/RBN_heterogeneous_paths_and_highest_order.tsv'

RBN_neutral_driver_df = pd.read_csv(RBN_save_file, sep='\t')


In [40]:
RBN_exist_networks = np.unique(RBN_neutral_driver_df.loc[RBN_neutral_driver_df['Existence']==1]['Network'])


In [41]:
print(len(RBN_exist_networks), len(np.unique(RBN_neutral_driver_df['Network'])))


11 120


In [42]:
highest_order_df = []
for network_name in np.unique(RBN_neutral_driver_df['Network']):
    highest_order = RBN_neutral_driver_df.loc[RBN_neutral_driver_df['Network']==network_name]['Highest_order'].max()
    highest_order_df.append([network_name, highest_order])

highest_order_df = pd.DataFrame(highest_order_df, columns=['Network', 'Highest_order'])

print(sum(highest_order_df['Highest_order']==1))
print(sum(highest_order_df['Highest_order']==2))
print(sum(highest_order_df['Highest_order']==3))
print(sum(highest_order_df['Highest_order']==4))


109
4
7
0
